In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, LSTM, Dropout, Reshape
from tensorflow.keras.layers import Bidirectional, TimeDistributed
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.models import load_model

%matplotlib inline

In [2]:
X_train, X_valid, X_test = np.load('./X_train.npy'), np.load('./X_valid.npy'), np.load('./X_test.npy')
y_train, y_valid, y_test = np.load('./Y_train.npy'), np.load('./Y_valid.npy'), np.load('./Y_test.npy')
print(X_train.shape, X_valid.shape, X_test.shape)
print(y_train.shape, y_valid.shape, y_test.shape)

(25711, 70, 4) (7093, 70, 4) (4306, 70, 4)
(25711, 70) (7093, 70) (4306, 70)


In [3]:
def matching(preds, true):
    total, match = 0, 0
    for p, t in zip(preds, true):
        if np.argmax(p) in np.nonzero(t)[0]:
            match += 1
        total += 1
    return match, total

In [4]:
K = 3
counts = np.zeros((2*K+1, 4))
for target, seq in zip(y_train, X_train):
    for bp in np.nonzero(target)[0]:
        if 0 > bp-K or bp+K+1 > seq.shape[0]: continue
        counts = counts + seq[bp-K: bp+K+1]
print(counts.T)

[[ 8658.  8037.  8984. 20929.  9468.  9994.  9281.]
 [12046.  8055. 11683.  6820. 11738. 11016. 10508.]
 [ 8100.  7269.  8965.  5728.  8041.  7817.  8519.]
 [10247. 15690.  9419.  5574.  9804. 10224. 10743.]]


In [5]:
new_model = load_model('./2layer.h5')
new_model.summary()

2021-11-16 15:16:44.821210: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-16 15:16:45.225836: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 612 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 70, 64)           9472      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 70, 64)           24832     
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 70, 1)            65        
 ibuted)                                                         
                                                                 
Total params: 34,369
Trainable params: 34,369
Non-trainable params: 0
_________________________________________________________________


In [6]:
test_loss, test_acc = new_model.evaluate(X_train, y_train, verbose=2)

2021-11-16 15:18:08.980428: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-16 15:18:08.980489: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:442 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
2021-11-16 15:18:08.984676: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-16 15:18:08.984692: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:442 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support
2021-11-16 15:18:08.988598: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2021-11-16 15:18:08.988613: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at matmul_op_impl.h:442 : INTERNAL: Attempting to perform BLAS

InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node sequential/bidirectional/forward_lstm/while/lstm_cell_1/MatMul
 (defined at /home/dwkim/.conda/envs/gpu/lib/python3.8/site-packages/keras/backend.py:2134)
]] [Op:__inference_test_function_2601]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/bidirectional/forward_lstm/while/lstm_cell_1/MatMul:
In[0] sequential/bidirectional/forward_lstm/while/lstm_cell_1/mul (defined at /home/dwkim/.conda/envs/gpu/lib/python3.8/site-packages/keras/layers/recurrent.py:2455)	
In[1] sequential/bidirectional/forward_lstm/while/lstm_cell_1/split (defined at /home/dwkim/.conda/envs/gpu/lib/python3.8/site-packages/keras/layers/recurrent.py:2464)

Operation defined at: (most recent call last)
>>>   File "/home/dwkim/.conda/envs/gpu/lib/python3.8/runpy.py", line 194, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   F